# Interpreting Breath Data for Health Screening

https://github.com/walmsley-lab/interpreting-breath-data-for-health-screening

## Introduction

Healthcare analytics has traditionally relied on structured clinical data such as diagnosis codes, procedures, and utilization records to evaluate outcomes, costs, and care quality at scale. Experience across provider-facing medical devices and payer-side analytics highlights both the power and the limits of this paradigm: while historical claims and encounter records support population-level insights, they often capture downstream consequences rather than early physiological signals.

As healthcare data sources expand, modalities such as wearables, low-cost biological assays, and chemical sensing technologies introduce opportunities for earlier and more subtle detection of disease processes. Among these, electronic nose (e-nose) technologies, arrays of broadly sensitive gas sensors, have shown promise in detecting disease-associated changes in volatile organic compounds (VOCs) linked to metabolic, respiratory, and oncologic conditions. Rather than identifying individual biomarkers, e-nose systems rely on multivariate response patterns that reflect shifts in underlying physiology.

This project explores whether deep learning methods can reliably extract such weak or early signals from breath-based sensor data using a principled, interpretable modeling pipeline. The objective is not to build a clinically deployable diagnostic tool, but to establish methodological foundations for representation learning, stability analysis, and interpretability in scent-based health screening. By grounding the analysis in realistic datasets and emphasizing transparency over complexity, this work aims to inform future applications of sensor-driven insights in value-based care and early-detection contexts.

## Data Selection and Rationale

Publicly available scent- and breath-based datasets vary widely in sensing modality, scale, and clinical grounding. Some datasets capture breath VOCs using gas chromatography–mass spectrometry (GC–MS), offering compound-level resolution but at the cost of small sample sizes, heavy preprocessing requirements, and limited reproducibility in open settings. Others rely on off-the-shelf electronic sensors that provide coarse but scalable signal patterns closer to what might be feasible in real-world deployments.

After surveying multiple candidates, including GC–MS breathomics datasets, electronic nose datasets for lung cancer screening, diabetes detection, and generic sensor drift studies, this project adopts the e-Nose Sensor Dataset for Predicting Human Diseases hosted on Kaggle as its primary data source. This dataset represents the largest publicly accessible collection of breath-related electronic sensor measurements paired with disease labels, comprising approximately 1,000 samples with balanced diabetic and control classes. Each sample consists of a single snapshot of six low-cost metal-oxide gas sensors with overlapping sensitivity profiles.

This choice reflects a deliberate tradeoff. While GC–MS datasets provide richer chemical specificity, their limited scale introduces instability and risks overfitting in deep learning workflows. In contrast, the selected e-nose dataset supports robust exploratory analysis, representation learning, and cross-validation within the constraints of an academic setting. The dataset’s structure built static, tabular, multivariate sensor responses mapped to health status, closely mirrors realistic screening scenarios where cost, noise, and scalability matter as much as signal strength.

## Problem Framing

Each observation in the dataset corresponds to a single breath or scent measurement captured by a six-sensor array and paired with an associated health label. The sensors themselves are not disease-specific; rather, each exhibits cross-sensitivity to overlapping subsets of volatile organic compounds (VOCs). As a result, informative signal arises from the collective response pattern across the sensor array rather than from any individual measurement.

Accordingly, the modeling task is framed as a screening and pattern-discovery problem. The objective is to assess whether disease-associated chemical signatures can be learned from multivariate sensor responses using deep learning methods that respect the static, low-dimensional structure of the data, rather than imposing inappropriate spatial or temporal assumptions.

In [ ]:
# ============================================================================
# CELL 1: SETUP AND IMPORTS
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.linalg import cholesky
import warnings
warnings.filterwarnings('ignore')

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, TensorDataset
    import torch.nn.functional as F
    
    # Set seeds for reproducibility
    SEED = 42
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.backends.cudnn.deterministic = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
except ImportError as e:
    print(f"Failed to import PyTorch: {e}")
    print("Please check your PyTorch installation")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Scikit-learn (for preprocessing and metrics)
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_curve, auc, accuracy_score, f1_score,
                             precision_score, recall_score, silhouette_score)
from sklearn.ensemble import RandomForestClassifier

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"NumPy version: {np.__version__}")
print(f"Random Seed: {SEED}")

In [ ]:
# ============================================================================
# CELL 2: DATA LOADING
# ============================================================================

import glob
import os

# Load data
data_source = "/kaggle/input/enose-sensor-dataset-for-predicting-human-diseases/enose_dataset_to_predict_human_disease.csv"
df = pd.read_csv(data_source)

# Note:
# During the data exploration phase a novel concept was introduced that we might use synthetic data.
# Doing so would allow us to develop, test, and demo our algorithm though insights would be lackluster.
# This data generation is possible by including a sensor correlation matrix capturing real signal correlations.
# Imagine, hydrocarbons might trigger TGS2610 and TGS2611 captured with high correlation score.
# This would be assembled using the Cholesky method. Demonstration omitted for brevity.

# Standardize column names
if 'Subjek' in df.columns:
    df = df.rename(columns={'Subjek': 'Diagnosis'})

# Identify sensor columns
exclude_cols = ['Time(s)', 'Number', 'Diagnosis']  # Fixed: 'Subjek' → 'Diagnosis' (post-rename)
sensor_cols = [col for col in df.columns if col not in exclude_cols 
               and df[col].dtype in ['float64', 'int64', 'float32', 'int32']]

# Data Quality Checks

print("="*60)
print("DATASET SUMMARY")
print("="*60)
print(f"Source: {data_source.split('/')[-1]}")  # Just filename
print(f"Shape: {df.shape[0]} samples × {df.shape[1]} columns")
print(f"Sensor Columns ({len(sensor_cols)}): {sensor_cols}")

print("\n" + "-"*60)
print("DIAGNOSIS DISTRIBUTION")
print("-"*60)
print(df['Diagnosis'].value_counts().to_string())
print(f"Total: {df['Diagnosis'].value_counts().sum()}")

print("\n" + "-"*60)
print("DATA QUALITY CHECKS")
print("-"*60)

# Check for nulls
null_counts = df[sensor_cols].isnull().sum()
total_nulls = null_counts.sum()
if total_nulls == 0:
    print("No missing values in sensor columns")
else:
    print(f"Missing values found: {total_nulls} total")
    print(null_counts[null_counts > 0].to_string())

# Check for non-numeric values in sensor columns
non_numeric_issues = []
for col in sensor_cols:
    if not pd.api.types.is_numeric_dtype(df[col]):
        non_numeric_issues.append(col)
    elif df[col].apply(lambda x: isinstance(x, (int, float)) or pd.isna(x)).all() == False:
        non_numeric_issues.append(col)

if len(non_numeric_issues) == 0:
    print("✓ All sensor columns are numeric")
else:
    print(f"⚠ Non-numeric data in: {non_numeric_issues}")

# Check for suspicious values (negatives, infinities)
has_negative = (df[sensor_cols] < 0).any().any()
has_inf = np.isinf(df[sensor_cols].values).any()

if not has_negative:
    print("✓ No negative sensor values")
else:
    neg_cols = [col for col in sensor_cols if (df[col] < 0).any()]
    print(f"⚠ Negative values in: {neg_cols}")

if not has_inf:
    print("✓ No infinite values")
else:
    print("⚠ Infinite values detected")

# Quick stats
print("\n" + "-"*60)
print("SENSOR STATISTICS")
print("-"*60)
print(df[sensor_cols].describe().round(1).to_string())

In [ ]:
# ============================================================================
# Sensor Distributions by Class
# ============================================================================

fig, axes = plt.subplots(1, 6, figsize=(18, 4), sharey=True)

colors = {'Normal': '#3498db', 'Diabetes': '#e74c3c'}

for ax, col in zip(axes, sensor_cols):
    for label in ['Normal', 'Diabetes']:
        subset = df[df['Diagnosis'] == label][col]
        ax.hist(
            subset,
            bins=30,
            density=True,
            alpha=0.6,
            color=colors[label],
            label=label
        )
    ax.set_title(col, fontsize=11)
    ax.set_xlabel('Reading')
    ax.grid(alpha=0.3)

axes[0].set_ylabel('Density')

# Shared legend
handles = [
    plt.Line2D([0], [0], color=colors['Normal'], lw=6, alpha=0.7),
    plt.Line2D([0], [0], color=colors['Diabetes'], lw=6, alpha=0.7)
]
labels = ['Normal', 'Diabetes']

fig.legend(
    handles,
    labels,
    loc='upper center',
    ncol=2,
    fontsize=11,
    frameon=False,
    bbox_to_anchor=(0.5, 1.12)
)

fig.suptitle(
    'Raw Sensor Distributions by Diagnosis',
    fontsize=14,
    fontweight='bold',
    y=1.22
)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Sensor-Level Signal and Overlap

Raw sensor distributions reveal that individual sensors exhibit partial but overlapping separation between normal and diabetic samples. Several sensors show clear shifts in central tendency or spread, while others display substantial overlap. This pattern indicates the presence of genuine disease-related signal at the single-sensor level, but also confirms that no individual sensor is sufficient for reliable classification. The observed overlap supports realistic expectations for biological data and motivates multivariate modeling approaches that integrate across sensors rather than relying on isolated biomarkers.

In [ ]:
# ============================================================================
# Signal and Structure
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Sensor Distributions (summary via effect sizes only)
effect_sizes = {}
for col in sensor_cols:
    n = df[df['Diagnosis']=='Normal'][col]
    d = df[df['Diagnosis']=='Diabetes'][col]
    pooled = np.sqrt((n.var() + d.var()) / 2)
    effect_sizes[col] = (d.mean() - n.mean()) / pooled

pd.Series(effect_sizes).sort_values(key=abs).plot.barh(
    ax=axes[0], color='steelblue', edgecolor='black'
)
axes[0].set_title('Sensor Effect Sizes (Cohen’s d)')
axes[0].axvline(0, color='black', lw=1)
axes[0].grid(axis='x', alpha=0.3)

# PCA Projection
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

for label, c in zip(class_names, ['#3498db', '#e74c3c']):
    m = df['Diagnosis'] == label
    axes[1].scatter(X_pca[m,0], X_pca[m,1], c=c, label=label,
                    alpha=0.6, edgecolors='white', s=50)

axes[1].set_title('PCA Projection')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
axes[1].legend()
axes[1].grid(alpha=0.3)

# t-SNE Projection
tsne = TSNE(n_components=2, random_state=SEED, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

for label, c in zip(class_names, ['#3498db', '#e74c3c']):
    m = df['Diagnosis'] == label
    axes[2].scatter(X_tsne[m,0], X_tsne[m,1], c=c, label=label,
                    alpha=0.6, edgecolors='white', s=50)

axes[2].set_title('t-SNE Projection')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Multivariate Structure and Separability

Sensor-level effect size analysis shows that discriminative power is unevenly distributed across the array. A small subset of sensors (notably TGS2611 and TGS2610) exhibits large effect sizes, while the remaining sensors contribute weaker but complementary signal. This confirms that disease information is present at the individual sensor level but concentrated rather than uniform, motivating representation learning approaches that can amplify informative sensors while attenuating noise.

Lower-dimensional projections clarify how these signals combine. PCA reveals partial class separation along the dominant variance directions, with the first two components accounting for the majority of total variance. Disease-related effects therefore align with global structure rather than residual noise dimensions, though substantial overlap remains, indicating incomplete linear separability in raw sensor space.

Nonlinear visualization with t-SNE further demonstrates coherent local structure, with samples of the same diagnosis forming compact neighborhoods despite global overlap. While t-SNE is used here only for qualitative assessment, the consistency of local clustering supports the presence of structured, multivariate disease signal. Together, these results show that breath sensor data exhibits low-dimensional, distributed, and biologically plausible structure, justifying downstream use of compact nonlinear representation learning rather than reliance on individual sensors or purely linear models.

In [ ]:
# ============================================================================
# Dimensionality and Interperability
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Cumulative Variance

pca_full = PCA().fit(X_scaled)
cum_var = np.cumsum(pca_full.explained_variance_ratio_)

axes[0].plot(range(1,len(cum_var)+1), cum_var, 'o-', lw=2)
axes[0].axhline(0.95, ls='--', color='gray')
axes[0].set_title('Cumulative Variance (PCA)')
axes[0].set_xlabel('Components')
axes[0].set_ylabel('Variance Explained')
axes[0].grid(alpha=0.3)

# Top-2 Sensor Scatter

top2 = sorted(effect_sizes, key=lambda k: abs(effect_sizes[k]), reverse=True)[:2]
for label, c in zip(class_names, ['#3498db', '#e74c3c']):
    sub = df[df['Diagnosis']==label]
    axes[1].scatter(sub[top2[0]], sub[top2[1]],
                    label=label, c=c, alpha=0.6, edgecolors='white')

axes[1].set_title('Top 2 Sensors (Raw Space)')
axes[1].set_xlabel(top2[0])
axes[1].set_ylabel(top2[1])
axes[1].legend()
axes[1].grid(alpha=0.3)

# Logistic Regression Coefficients

lr = LogisticRegression(max_iter=1000)
lr.fit(X_scaled, y_encoded)
coef_df = pd.Series(lr.coef_[0], index=sensor_cols).sort_values()

coef_df.plot.barh(ax=axes[2], color=['#3498db' if v<0 else '#e74c3c' for v in coef_df])
axes[2].set_title('Logistic Regression Coefficients')
axes[2].axvline(0, color='black')
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### Low-Dimensional and Linear Structure

Cumulative variance analysis indicates strong redundancy in the breath sensor signals: the first two principal components capture the majority of total variance, with additional components contributing diminishing returns. This low intrinsic dimensionality supports the use of compact latent representations rather than high-capacity models operating directly in raw sensor space.

Projection onto the two most discriminative sensors (TGS2611 and TGS2610) shows that meaningful class separation is already present in the original measurements. Although overlap remains, disease and normal samples occupy distinct regions, demonstrating that downstream model performance is grounded in genuine sensor signal rather than artifacts introduced by representation learning.

A regularized logistic regression baseline identifies a similar subset of influential sensors, with coefficient magnitudes aligning closely with univariate effect sizes. This agreement across univariate statistics, raw-space projections, and linear multivariate modeling suggests that more expressive models primarily refine existing structure rather than hallucinate separability. Residual overlap in linear projections, together with known nonlinear responses of metal-oxide sensors, motivates the use of nonlinear representation learning to capture higher-order structure while preserving interpretability.

In [ ]:
# ============================================================================
# Interperability and Structual Change
# ============================================================================

# Top-2 Sensor Scatter

top2 = sorted(effect_sizes.items(), key=lambda x: abs(x[1]), reverse=True)[:2]
sx, sy = top2[0][0], top2[1][0]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for label in class_names:
    subset = df[df["Diagnosis"] == label]
    axes[0].scatter(subset[sx], subset[sy],
                    label=label, alpha=0.6, color=colors[label])

axes[0].set_xlabel(sx)
axes[0].set_ylabel(sy)
axes[0].set_title("Top-2 Sensor Scatter")
axes[0].legend()
axes[0].grid(alpha=0.3)

# Logistic Regression Coefficients

lr = LogisticRegression(max_iter=1000, random_state=SEED)
lr.fit(X_scaled, y_encoded)
coef_df = pd.DataFrame({"Sensor": sensor_cols, "Coefficient": lr.coef_[0]})
coef_df = coef_df.sort_values("Coefficient")

axes[1].barh(coef_df["Sensor"], coef_df["Coefficient"],
             color=["#e74c3c" if c > 0 else "#3498db" for c in coef_df["Coefficient"]])
axes[1].axvline(0, color="black", linewidth=0.8)
axes[1].set_title("Logistic Regression Coefficients")

# Correlation Difference Heatmap

corr_diff = df_disease.corr() - df_normal.corr()
mask = np.triu(np.ones_like(corr_diff, dtype=bool), k=1)

sns.heatmap(corr_diff, mask=mask, cmap="RdBu_r", center=0,
            annot=True, fmt=".2f", square=True, ax=axes[2])
axes[2].set_title("Correlation Shift (Disease − Normal)")

plt.tight_layout()
plt.show()

### Inter-Sensor Structure and Attribution

Joint analysis of raw sensor scatter, linear attribution, and correlation shifts triangulates the source of separability observed in downstream models. Projection onto TGS2611 and TGS2610, the most discriminative sensors, already reveals meaningful class structure in raw measurements. While overlap remains, samples from each class occupy distinct regions, confirming that disease signal exists prior to representation learning.

Logistic regression reinforces this conclusion: sensors with large univariate effect sizes receive the strongest linear weights, indicating agreement between marginal statistics and multivariate attribution. This consistency suggests that more complex models primarily refine existing structure rather than manufacturing separability.

Beyond marginal effects, correlation difference analysis shows that disease alters inter-sensor relationships. Several sensor pairs exhibit substantial shifts in correlation magnitude between normal and disease groups, indicating changes in joint response behavior rather than isolated sensor deviations. Such shifts may reflect altered metabolic coupling or sensor response regimes under disease conditions.

Taken together, these findings indicate that classification performance arises from structured changes in sensor behavior, both in individual responses and their interactions, motivating latent representations that can capture relational structure rather than treating sensors as independent channels.

## Architecture

#### Convolutional Neural Networks

Convolutional neural networks assume that neighboring inputs share semantic meaning and that learned feature detectors can be applied consistently across positions (translational invariance). This inductive bias is highly effective for images, spectrograms, and other data with meaningful spatial structure.

Although a vector of sensor values can technically be processed using one-dimensional convolutions, doing so implicitly assumes that adjacent sensors form reusable local patterns. In this dataset, sensor adjacency is arbitrary and does not correspond to physical proximity or functional similarity. Sliding convolutional kernels across unordered sensor measurements therefore does not preserve semantic meaning and provides no clear advantage over fully connected layers. As a result, CNN-based architectures would introduce inductive bias unsupported by the data.

CNNs would become appropriate if sensor measurements were represented as time–frequency images, derived from dense spatial sensor arrays, or otherwise embedded in a structured spatial domain.

#### Recurrent Neural Networks

Recurrent neural networks, including LSTM variants, are designed to model ordered sequences in which the interpretation of a given input depends on previous elements. In the present dataset, each observation represents an independent measurement rather than a sequence, and there is no intrinsic temporal ordering among sensor values.

Feeding sensor readings into an RNN would therefore impose artificial sequential structure, encouraging the model to learn dependencies driven by arbitrary feature ordering rather than meaningful dynamics. This additional complexity offers no benefit in the absence of temporal information and risks learning spurious relationships.

RNN-based approaches would be well suited to extensions of this work involving continuous breath monitoring, repeated measurements over time, or modeling of sensor response dynamics within individual breath samples.

#### Feedforward Neural Networks

Given these considerations, feedforward neural networks were selected as the primary modeling approach. Feedforward architectures impose minimal structural assumptions, treating inputs as unordered feature vectors and allowing interactions among sensors to be learned directly from the data. This flexibility is particularly appropriate when features lack inherent spatial or temporal structure and when feature relationships are not known a priori.

The exploratory analysis supports this selection. Logistic regression achieves strong baseline performance, indicating approximate linear separability, while correlation analysis reveals inter-sensor dependencies that more expressive models can exploit. Feedforward networks provide sufficient capacity to capture these interactions without introducing unnecessary inductive bias.

#### Design Choices

Dataset size and structure can further guide architectural decisions. With six input features and a limited sample size (~10³ observations), deep networks are unlikely to provide additional representational benefit and may increase overfitting risk. Imagine the configuration of the form Input (6) → Hidden (32–64) → Hidden (16–32) → Output (1).  Shallow architectures with two or three hidden layers strike an appropriate balance between expressiveness and generalization.

Wider early layers capture feature interactions, while progressively narrower layers compress toward the decision boundary. Principal component analysis indicates that around three latent dimensions will capture the majority of variance, suggesting substantial feature redundancy. To address this we can employ Bottleneck arcitecture. Imagine the configuration Input (6) → Encoder (32 → 16) → Bottleneck (3–4) → Classifier / Decoder. The bottleneck constrains model capacity, promotes regularization, and enables interpretability by encouraging compact latent representations that may correspond to underlying physiological factors. Given the modest dataset size, multiple regularization strategies were applied, including dropout, L2 weight decay, early stopping based on validation performance, and batch normalization to stabilize training.

Both of these complementary models variants will be explored. The first, a supervised feedforward classifier optimized directly for disease discrimination.  The other was an autoencoder-based model that learns unsupervised latent structure prior to classification. This comparison assesses whether reconstruction-based representations improve robustness and generalization. These architectures aligns model assumptions with the structure of the data. Convolutional and recurrent architectures rely on spatial or sequential dependencies that are absent in unordered, static sensor measurements. Feedforward networks, by contrast, impose minimal inductive bias and are well suited for tabular data, allowing sensor interactions to be learned directly. The inclusion of a bottleneck layer further reflects the empirically observed low intrinsic dimensionality, enabling effective regularization and interpretability while avoiding unnecessary model complexity.

In [ ]:
# ============================================================================
# CELL 5: PYTORCH MODEL DEFINITIONS
# ============================================================================

# Autoencoder for Unsupervised Feature Learning + Reconstruction

class Autoencoder(nn.Module):
    """
    Autoencoder for unsupervised feature learning and reconstruction.
    
    Architecture:
        Encoder: input_dim → 32 → 16 → bottleneck_dim
        Decoder: bottleneck_dim → 16 → 32 → input_dim
    
    Bottleneck provides compressed interpretable features.
    Reconstruction loss ensures features capture essential info.
    """
    
    def __init__(self, input_dim, bottleneck_dim=8, dropout_rate=0.3):
        super(Autoencoder, self).__init__()
        
        self.bottleneck_dim = bottleneck_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(16, bottleneck_dim),
            nn.BatchNorm1d(bottleneck_dim),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(16, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(32, input_dim)
        )
    
    def encode(self, x):
        return self.encoder(x)
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        z = self.encode(x)
        reconstructed = self.decode(z)
        return reconstructed, z


# Classifier with Bottleneck

class BottleneckClassifier(nn.Module):
    """
    Feedforward classifier with bottleneck layer for interpretable features.
    
    Architecture:
        input_dim → 32 → 16 → bottleneck_dim → 1 (sigmoid)
    
    The bottleneck forces information compression, making features interpretable.
    """
    
    def __init__(self, input_dim, bottleneck_dim=8, dropout_rate=0.3):
        super(BottleneckClassifier, self).__init__()
        
        self.bottleneck_dim = bottleneck_dim
        
        # Feature extraction layers
        self.features = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(16, bottleneck_dim),
            nn.BatchNorm1d(bottleneck_dim),
            nn.ReLU()
        )
        
        # Classification head
        self.classifier = nn.Linear(bottleneck_dim, 1)
    
    def get_bottleneck_features(self, x):
        return self.features(x)
    
    def forward(self, x):
        features = self.features(x)
        logits = self.classifier(features)
        return logits, features


# Combined Model (Autoencoder + Classifier)

class ENoseNet(nn.Module):
    """
    Combined model with autoencoder (reconstruction) and classifier.
    
    Enables:
    1. Unsupervised pretraining via reconstruction
    2. Supervised fine-tuning for classification
    3. Both reconstruction and classification visualization
    """
    
    def __init__(self, input_dim, bottleneck_dim=8, dropout_rate=0.3):
        super(ENoseNet, self).__init__()
        
        self.bottleneck_dim = bottleneck_dim
        
        # Shared encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(16, bottleneck_dim),
            nn.BatchNorm1d(bottleneck_dim),
            nn.ReLU()
        )
        
        # Decoder for reconstruction
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            nn.Linear(16, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
            nn.Linear(32, input_dim)
        )
        
        # Classifier head
        self.classifier = nn.Linear(bottleneck_dim, 1)
    
    def encode(self, x):
        return self.encoder(x)
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x, return_reconstruction=False):
        z = self.encode(x)
        logits = self.classifier(z)
        
        if return_reconstruction:
            reconstruction = self.decode(z)
            return logits, z, reconstruction
        
        return logits, z


# Print architecture
print("\nModel Architecture:")
print("─"*70)
model_example = ENoseNet(input_dim=6, bottleneck_dim=8)
print(model_example)

# Parameter count
total_params = sum(p.numel() for p in model_example.parameters())
trainable_params = sum(p.numel() for p in model_example.parameters() if p.requires_grad)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

In [ ]:
# ============================================================================
# CELL 6: DATA PREPARATION FOR PYTORCH
# ============================================================================

print("="*70)
print("5. DATA PREPARATION")
print("="*70)

# Prepare numpy arrays

X = df[sensor_cols].values.astype(np.float32)
y = le.fit_transform(df['Diagnosis'].values).astype(np.float32)

# Train/Val/Test split (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=SEED, stratify=y_temp
)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_val_scaled = scaler.transform(X_val).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

print(f"\nData Splits:")
print(f"  Training:   {len(X_train)} samples ({len(X_train)/len(X):.0%})")
print(f"  Validation: {len(X_val)} samples ({len(X_val)/len(X):.0%})")
print(f"  Test:       {len(X_test)} samples ({len(X_test)/len(X):.0%})")

# Create PyTorch datasets and dataloaders

# Convert to tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nDataLoaders created with batch_size={BATCH_SIZE}")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

In [ ]:
# ============================================================================
# CELL 7: TRAINING FUNCTIONS
# ============================================================================

# Training function with both reconstruction and classification losses
def train_epoch(model, train_loader, optimizer, device, alpha_recon=0.3):
    """
    Train for one epoch with combined loss:
    L_total = L_classification + alpha * L_reconstruction
    """
    model.train()
    total_loss = 0
    total_cls_loss = 0
    total_recon_loss = 0
    correct = 0
    total = 0
    
    criterion_cls = nn.BCEWithLogitsLoss()
    criterion_recon = nn.MSELoss()
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        logits, features, reconstruction = model(X_batch, return_reconstruction=True)
        
        # Losses
        loss_cls = criterion_cls(logits, y_batch)
        loss_recon = criterion_recon(reconstruction, X_batch)
        loss = loss_cls + alpha_recon * loss_recon
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item() * X_batch.size(0)
        total_cls_loss += loss_cls.item() * X_batch.size(0)
        total_recon_loss += loss_recon.item() * X_batch.size(0)
        
        predictions = (torch.sigmoid(logits) > 0.5).float()
        correct += (predictions == y_batch).sum().item()
        total += y_batch.size(0)
    
    return {
        'loss': total_loss / total,
        'cls_loss': total_cls_loss / total,
        'recon_loss': total_recon_loss / total,
        'accuracy': correct / total
    }


def evaluate(model, data_loader, device, alpha_recon=0.3):
    """Evaluate model on given data loader"""
    model.eval()
    total_loss = 0
    total_cls_loss = 0
    total_recon_loss = 0
    correct = 0
    total = 0
    
    all_probs = []
    all_labels = []
    
    criterion_cls = nn.BCEWithLogitsLoss()
    criterion_recon = nn.MSELoss()
    
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            logits, features, reconstruction = model(X_batch, return_reconstruction=True)
            
            loss_cls = criterion_cls(logits, y_batch)
            loss_recon = criterion_recon(reconstruction, X_batch)
            loss = loss_cls + alpha_recon * loss_recon
            
            total_loss += loss.item() * X_batch.size(0)
            total_cls_loss += loss_cls.item() * X_batch.size(0)
            total_recon_loss += loss_recon.item() * X_batch.size(0)
            
            probs = torch.sigmoid(logits)
            predictions = (probs > 0.5).float()
            correct += (predictions == y_batch).sum().item()
            total += y_batch.size(0)
            
            all_probs.extend(probs.cpu().numpy().flatten())
            all_labels.extend(y_batch.cpu().numpy().flatten())
    
    return {
        'loss': total_loss / total,
        'cls_loss': total_cls_loss / total,
        'recon_loss': total_recon_loss / total,
        'accuracy': correct / total,
        'probs': np.array(all_probs),
        'labels': np.array(all_labels)
    }


def train_model(model, train_loader, val_loader, optimizer, scheduler, device,
                n_epochs=200, patience=20, alpha_recon=0.3):
    """
    Full training loop with early stopping
    """
    history = {
        'train_loss': [], 'val_loss': [],
        'train_cls_loss': [], 'val_cls_loss': [],
        'train_recon_loss': [], 'val_recon_loss': [],
        'train_acc': [], 'val_acc': [],
        'lr': []
    }
    
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    
    for epoch in range(n_epochs):
        # Train
        train_metrics = train_epoch(model, train_loader, optimizer, device, alpha_recon)
        
        # Validate
        val_metrics = evaluate(model, val_loader, device, alpha_recon)
        
        # Learning rate scheduling
        if scheduler is not None:
            scheduler.step(val_metrics['loss'])
        
        # Record history
        history['train_loss'].append(train_metrics['loss'])
        history['val_loss'].append(val_metrics['loss'])
        history['train_cls_loss'].append(train_metrics['cls_loss'])
        history['val_cls_loss'].append(val_metrics['cls_loss'])
        history['train_recon_loss'].append(train_metrics['recon_loss'])
        history['val_recon_loss'].append(val_metrics['recon_loss'])
        history['train_acc'].append(train_metrics['accuracy'])
        history['val_acc'].append(val_metrics['accuracy'])
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        # Early stopping
        if val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Print progress
        if (epoch + 1) % 20 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:3d}/{n_epochs} | "
                  f"Train Loss: {train_metrics['loss']:.4f} | "
                  f"Val Loss: {val_metrics['loss']:.4f} | "
                  f"Train Acc: {train_metrics['accuracy']:.4f} | "
                  f"Val Acc: {val_metrics['accuracy']:.4f}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return history

In [ ]:
# ============================================================================
# VISUALIZATION: MODEL CONVERGENCE / OPTIMIZER COMPARISON (LEAN VERSION)
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Total Loss (Classification + Reconstruction)

axes[0].plot(history_sgd['train_loss'], label='SGD Train', linewidth=2, color='blue')
axes[0].plot(history_sgd['val_loss'], label='SGD Val', linewidth=2, linestyle='--', color='blue')
axes[0].plot(history_adam['train_loss'], label='Adam Train', linewidth=2, color='red')
axes[0].plot(history_adam['val_loss'], label='Adam Val', linewidth=2, linestyle='--', color='red')

axes[0].set_title('Total Loss (Train vs Validation)', fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)


# Training / Validation Accuracy

axes[1].plot(history_sgd['train_acc'], label='SGD Train', linewidth=2, color='blue')
axes[1].plot(history_sgd['val_acc'], label='SGD Val', linewidth=2, linestyle='--', color='blue')
axes[1].plot(history_adam['train_acc'], label='Adam Train', linewidth=2, color='red')
axes[1].plot(history_adam['val_acc'], label='Adam Val', linewidth=2, linestyle='--', color='red')

axes[1].set_title('Training and Validation Accuracy', fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)


# Test Set Performance

sgd_metrics = evaluate(model_sgd, test_loader, device)
adam_metrics = evaluate(model_adam, test_loader, device)

metrics_comparison = {
    'SGD': [
        sgd_metrics['accuracy'],
        auc(*roc_curve(sgd_metrics['labels'], sgd_metrics['probs'])[:2])
    ],
    'Adam': [
        adam_metrics['accuracy'],
        auc(*roc_curve(adam_metrics['labels'], adam_metrics['probs'])[:2])
    ]
}

x = np.arange(2)
width = 0.35

axes[2].bar(
    x - width / 2, metrics_comparison['SGD'],
    width, label='SGD', color='blue', edgecolor='black'
)
axes[2].bar(
    x + width / 2, metrics_comparison['Adam'],
    width, label='Adam', color='red', edgecolor='black'
)

axes[2].set_xticks(x)
axes[2].set_xticklabels(['Accuracy', 'AUC'])
axes[2].set_ylim(0.9, 1.01)
axes[2].set_ylabel('Score')
axes[2].set_title('Test Set Performance', fontweight='bold')
axes[2].legend()

# Value labels
for i, v in enumerate(metrics_comparison['SGD']):
    axes[2].text(i - width/2, v + 0.005, f'{v:.3f}', ha='center', fontsize=10)
for i, v in enumerate(metrics_comparison['Adam']):
    axes[2].text(i + width/2, v + 0.005, f'{v:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('fig4_training_convergence.png', dpi=150, bbox_inches='tight')
plt.show()


Both SGD with momentum and Adam converge reliably to high-performing solutions. Adam reduces loss more quickly early in training, while SGD converges more smoothly with closely aligned training and validation losses throughout. Classification loss drops rapidly, indicating that the decision boundary is learned early, consistent with the strong class separability observed during exploratory analysis. Reconstruction loss declines more gradually as the latent representation is refined, reflecting its broader objective. Identical test performance suggests that overall gains are driven more by model capacity and data separability than by optimizer choice. SGD was therefore selected due to its slightly lower validation loss and more stable late-stage convergence.

In [ ]:
# ============================================================================
# CELL 9: RECONSTRUCTION VISUALIZATION
# ============================================================================

model.eval()

# Get reconstructions for test set
with torch.no_grad():
    X_test_tensor_device = X_test_tensor.to(device)
    _, features_test, reconstructions = model(
        X_test_tensor_device, return_reconstruction=True
    )

    reconstructions = reconstructions.cpu().numpy()
    features_test = features_test.cpu().numpy()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Reconstruction error by class

recon_errors = np.mean((X_test_scaled - reconstructions) ** 2, axis=1)
recon_errors_normal = recon_errors[y_test == 0]
recon_errors_disease = recon_errors[y_test == 1]

axes[0].hist(
    recon_errors_normal, bins=30, alpha=0.6,
    label=class_names[0], color='green',
    edgecolor='black', density=True
)
axes[0].hist(
    recon_errors_disease, bins=30, alpha=0.6,
    label=class_names[1], color='red',
    edgecolor='black', density=True
)

axes[0].set_title('Reconstruction Error by Class', fontweight='bold')
axes[0].set_xlabel('Mean Squared Error')
axes[0].set_ylabel('Density')
axes[0].legend()
axes[0].grid(True, alpha=0.3)


# Original vs reconstructed (single sensor)

sensor_idx = 0  # first sensor
axes[1].scatter(
    X_test_scaled[:, sensor_idx],
    reconstructions[:, sensor_idx],
    c=y_test, cmap='RdYlGn',
    alpha=0.6, edgecolors='black', linewidths=0.5
)

axes[1].plot([-3, 3], [-3, 3], 'k--', linewidth=2, label='Perfect reconstruction')

axes[1].set_title(
    f'{sensor_cols[sensor_idx]}: Original vs Reconstructed',
    fontweight='bold'
)
axes[1].set_xlabel(f'Original {sensor_cols[sensor_idx]}')
axes[1].set_ylabel(f'Reconstructed {sensor_cols[sensor_idx]}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)


# Reconstruction R² by sensor

r2_scores = []
for i in range(len(sensor_cols)):
    ss_res = np.sum((X_test_scaled[:, i] - reconstructions[:, i]) ** 2)
    ss_tot = np.sum((X_test_scaled[:, i] - X_test_scaled[:, i].mean()) ** 2)
    r2 = 1 - ss_res / ss_tot
    r2_scores.append(r2)

axes[2].bar(
    sensor_cols, r2_scores,
    color='forestgreen', edgecolor='black'
)
axes[2].axhline(
    y=0.9, color='red', linestyle='--',
    alpha=0.7, label='0.9 threshold'
)

axes[2].set_title('Reconstruction R² by Sensor', fontweight='bold')
axes[2].set_ylabel('R² Score')
axes[2].tick_params(axis='x', rotation=45)
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('fig5_reconstruction.png', dpi=150, bbox_inches='tight')
plt.show()


Reconstruction analysis indicates that the autoencoder’s bottleneck preserves the majority of input signal without introducing class-dependent distortion. Reconstruction error distributions for diabetic and normal samples overlap substantially, with similar mean reconstruction error, confirming that neither class is preferentially represented during encoding. At the sensor level, reconstruction quality remains consistently high, with per-sensor explained variance exceeding ~90%, supporting earlier PCA findings of low intrinsic dimensionality. These results demonstrate that compressing six sensor measurements into a low-dimensional latent space retains meaningful structure rather than collapsing inputs toward class averages. While balanced reconstruction alone does not induce class separation, it confirms that downstream classification performance arises from the geometry of the learned feature space rather than from differential reconstruction artifacts, supporting the use of the bottleneck representation as a stable and unbiased basis for disease classification.

In [ ]:
# ============================================================================
# CELL 10: VISUALIZATION - ROC CURVE & CONFUSION MATRIX
# ============================================================================

# Get final predictions
test_metrics = evaluate(model, test_loader, device)
y_pred_prob = test_metrics['probs']
y_pred = (y_pred_prob > 0.5).astype(int)
y_true = test_metrics['labels'].astype(int)

# Create layout
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve

fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

axes[0].plot(fpr, tpr, lw=3, color='darkorange',
             label=f'ROC Curve (AUC = {roc_auc:.3f})')
axes[0].fill_between(fpr, tpr, alpha=0.3, color='orange')
axes[0].plot([0, 1], [0, 1], '--', color='navy', lw=2,
             label='Random (AUC = 0.500)')

# Optimal threshold marker
optimal_idx = np.argmax(tpr - fpr)
axes[0].scatter(
    fpr[optimal_idx], tpr[optimal_idx],
    color='red', s=80, zorder=5,
    label=f'Optimal threshold = {thresholds[optimal_idx]:.2f}'
)

axes[0].set_xlim([0, 1])
axes[0].set_ylim([0, 1.05])
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Confusion Matrix

cm = confusion_matrix(y_true, y_pred)

sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    ax=axes[1],
    xticklabels=class_names,
    yticklabels=class_names,
    annot_kws={'size': 14},
    cbar=False
)

axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_title('Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('fig6_roc_confusion.png', dpi=150, bbox_inches='tight')
plt.show()


Classification results show complete separation between diabetic and normal samples on the held-out test set, achieving perfect accuracy, precision, recall, F1 score, and ROC–AUC (all 1.0). The ROC curve rises vertically to perfect sensitivity at zero false positive rate, indicating deterministic separability rather than performance driven by threshold tuning. This outcome is consistent with earlier latent space and reconstruction analyses that revealed minimal class overlap. Nevertheless, perfect performance on biomedical data warrants cautious interpretation, as it may reflect limited sample diversity or test set size rather than true population-level generalizability. External validation on independent cohorts, potentially collected using different hardware, measurement protocols, or patient demographics, would be required to assess robustness under distributional shift and establish clinical applicability.

In [ ]:
# ============================================================================
# CELL 11: FEATURE INTERPRETATION (REFINED)
# ============================================================================

# Extract features for all data (inference only)

model.eval()
X_all_scaled = scaler.transform(X).astype(np.float32)
X_all_tensor = torch.tensor(X_all_scaled, dtype=torch.float32).to(device)

with torch.no_grad():
    _, features_all, _ = model(X_all_tensor, return_reconstruction=True)
    features_all = features_all.cpu().numpy()

# Visulizing Raw vs Learned Feature Space (PCA)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Raw sensor space
pca_raw = PCA(n_components=2, svd_solver='full', random_state=42)
X_raw_pca = pca_raw.fit_transform(X_all_scaled)

axes[0].scatter(X_raw_pca[y_encoded == 0, 0], X_raw_pca[y_encoded == 0, 1],
                c='green', label=class_names[0], alpha=0.6, edgecolors='black', s=50)
axes[0].scatter(X_raw_pca[y_encoded == 1, 0], X_raw_pca[y_encoded == 1, 1],
                c='red', label=class_names[1], alpha=0.6, edgecolors='black', s=50)
axes[0].set_xlabel(f'PC1 ({pca_raw.explained_variance_ratio_[0]:.1%})')
axes[0].set_ylabel(f'PC2 ({pca_raw.explained_variance_ratio_[1]:.1%})')
axes[0].set_title('Raw Sensor Space (PCA)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Learned bottleneck feature space
pca_learned = PCA(n_components=2, svd_solver='full', random_state=42)
X_learned_pca = pca_learned.fit_transform(features_all)

axes[1].scatter(X_learned_pca[y_encoded == 0, 0], X_learned_pca[y_encoded == 0, 1],
                c='green', label=class_names[0], alpha=0.6, edgecolors='black', s=50)
axes[1].scatter(X_learned_pca[y_encoded == 1, 0], X_learned_pca[y_encoded == 1, 1],
                c='red', label=class_names[1], alpha=0.6, edgecolors='black', s=50)
axes[1].set_xlabel(f'Learned PC1 ({pca_learned.explained_variance_ratio_[0]:.1%})')
axes[1].set_ylabel(f'Learned PC2 ({pca_learned.explained_variance_ratio_[1]:.1%})')
axes[1].set_title('Learned Feature Space (PCA)', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Feature activations by class
feature_names = [f'F{i+1}' for i in range(bottleneck_dim)]
feature_means = pd.DataFrame({
    'Feature': feature_names,
    class_names[0]: features_all[y_encoded == 0].mean(axis=0),
    class_names[1]: features_all[y_encoded == 1].mean(axis=0)
})

x = np.arange(bottleneck_dim)
width = 0.35
axes[2].bar(x - width/2, feature_means[class_names[0]], width,
            label=class_names[0], color='green', edgecolor='black')
axes[2].bar(x + width/2, feature_means[class_names[1]], width,
            label=class_names[1], color='red', edgecolor='black')
axes[2].set_xlabel('Bottleneck Feature')
axes[2].set_ylabel('Mean Activation')
axes[2].set_title('Feature Activations by Class', fontsize=14, fontweight='bold')
axes[2].set_xticks(x)
axes[2].set_xticklabels(feature_names)
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('fig7_feature_separation.png', dpi=150, bbox_inches='tight')
plt.show()

# Quantify class separation improvement (silhouette score)

sil_raw = silhouette_score(X_all_scaled, y_encoded)
sil_learned = silhouette_score(features_all, y_encoded)

print("\nClass Separation Comparison:")
print(f"  Raw sensor space silhouette:     {sil_raw:.3f}")
print(f"  Learned feature space silhouette:{sil_learned:.3f}")
print(f"  Absolute improvement:            {sil_learned - sil_raw:.3f}")

# Input-to-feature correlation analysis

input_feature_corr = np.zeros((len(sensor_cols), bottleneck_dim))
for i in range(len(sensor_cols)):
    for j in range(bottleneck_dim):
        input_feature_corr[i, j] = np.corrcoef(X_all_scaled[:, i], features_all[:, j])[0, 1]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(input_feature_corr, annot=True, fmt='.2f',
            cmap='RdBu_r', center=0, vmin=-1, vmax=1,
            xticklabels=feature_names, yticklabels=sensor_cols, ax=axes[0])
axes[0].set_title('Input–Feature Correlation', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Bottleneck Feature')
axes[0].set_ylabel('Input Sensor')

# Gradient-based sensor importance (class-conditional)

X_tensor_grad = torch.tensor(X_all_scaled, dtype=torch.float32, requires_grad=True).to(device)
logits, _ = model(X_tensor_grad)

# Focus gradients on disease class
loss = logits[y_encoded == 1].sum()
loss.backward()

gradient_importance = X_tensor_grad.grad.abs().mean(dim=0).cpu().numpy()
gradient_importance /= gradient_importance.max()

axes[1].barh(sensor_cols, gradient_importance,
             color='steelblue', edgecolor='black')
axes[1].set_xlabel('Relative Importance (Gradient)')
axes[1].set_title('Sensor Importance (Gradient-based)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('fig8_feature_interpretation.png', dpi=150, bbox_inches='tight')
plt.show()

# Feature naming (interpretable summary)

biomarker_map = {
    'TGS2610': ('Acetone, Butane', 'Diabetes, Ketoacidosis'),
    'TGS2611': ('Methane, Acetone', 'Diabetes, GI disorders'),
    'TGS2600': ('Hydrogen, Ethanol', 'General metabolic'),
    'TGS2602': ('Ammonia, H₂S, VOCs', 'Kidney and liver disease'),
    'TGS826':  ('Ammonia, Amines', 'Kidney disease, H. pylori'),
    'TGS2620': ('Ethanol, Solvents', 'Liver disease, Diabetes'),
}

for j in range(bottleneck_dim):
    corrs = input_feature_corr[:, j]
    top_idx = np.argsort(np.abs(corrs))[::-1][:2]
    top_sensors = [sensor_cols[i] for i in top_idx]
    top_corrs = [corrs[i] for i in top_idx]

    class_diff = features_all[y_encoded == 1, j].mean() - features_all[y_encoded == 0, j].mean()
    direction = "↑ Disease" if class_diff > 0.1 else "↓ Disease" if class_diff < -0.1 else "≈ Similar"

    compounds = biomarker_map.get(top_sensors[0], ('Unknown', 'Unknown'))[0]

    print(f"\n  Feature F{j+1}:")
    print(f"    Dominant sensors: {top_sensors[0]} (r={top_corrs[0]:.2f}), "
          f"{top_sensors[1]} (r={top_corrs[1]:.2f})")
    print(f"    Putative compounds (sensor sensitivity): {compounds}")
    print(f"    Class pattern: {direction} (Δ={class_diff:.2f})")


To evaluate whether the learned bottleneck representation captures meaningful and interpretable structure, we examined class separability, sensor associations, and feature activation patterns within the latent space. Dimensionality reduction of the bottleneck features reveals substantially improved class separation compared to the raw sensor space, consistent with the observed increase in silhouette score. This indicates that the encoder organizes samples into a geometry that more explicitly reflects disease status, rather than merely compressing input variance.

Analysis of bottleneck feature activations shows that only a subset of latent dimensions exhibits strong class-dependent behavior, while others remain relatively balanced across classes. This sparsity suggests that disease-relevant information is concentrated in specific axes of the latent space rather than diffusely encoded. Correlation analysis between bottleneck features and input sensors further indicates that latent dimensions reflect structured combinations of sensor responses, rather than dominance by individual sensors alone.

To support interpretability, gradient-based sensitivity analysis was used to identify input sensors most influential for disease prediction. These importance patterns are consistent with both linear baseline coefficients and latent-space structure, reinforcing the notion that the model’s decisions rely on physiologically meaningful signal variations captured by the sensor array.

Finally, we present putative interpretations of bottleneck features based on their strongest statistical associations with individual sensors and known sensor sensitivity profiles. These interpretations are hypothesis-generating and do not constitute direct identification of specific breath biomarkers. Rather, they provide an interpretable bridge between learned representations and known chemical sensitivities of the sensor suite, offering insight into how latent dimensions may correspond to dominant response patterns. The balanced reconstruction quality across classes and consistent feature geometry support the use of the bottleneck representation as a stable, unbiased feature space for downstream classification.

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

accs, aucs = [], []

for train_idx, val_idx in kfold.split(X, y_encoded):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X[train_idx]).astype(np.float32)
    X_val   = scaler.transform(X[val_idx]).astype(np.float32)

    y_train = y_encoded[train_idx].astype(np.float32)
    y_val   = y_encoded[val_idx].astype(np.float32)

    train_loader = DataLoader(
        TensorDataset(torch.tensor(X_train), torch.tensor(y_train).unsqueeze(1)),
        batch_size=32, shuffle=True
    )
    val_loader = DataLoader(
        TensorDataset(torch.tensor(X_val), torch.tensor(y_val).unsqueeze(1)),
        batch_size=32, shuffle=False
    )

    model = ENoseNet(input_dim=len(sensor_cols), bottleneck_dim=8).to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)

    train_model(model, train_loader, val_loader, optimizer, None, device,
                n_epochs=100, patience=15, alpha_recon=0.3)

    metrics = evaluate(model, val_loader, device)
    auc_val = auc(*roc_curve(metrics['labels'], metrics['probs'])[:2])

    accs.append(metrics['accuracy'])
    aucs.append(auc_val)

print("\n" + "─"*50)
print("Cross-Validation Summary (5-fold)")
print("─"*50)
print(f"Accuracy: {np.mean(accs):.3f} ± {np.std(accs):.3f}")
print(f"AUC:      {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")
print("─"*50)

Stratified five-fold cross-validation was conducted to assess the stability of the full modeling pipeline with respect to data partitioning. Preprocessing and model training were performed independently within each fold to prevent information leakage. Performance remained consistent across all folds, indicating that model behavior is not driven by a favorable train–test split but reflects structure present throughout the dataset. While cross-validation does not substitute for external validation on independent cohorts, these results provide evidence of internal stability and robustness of the learned decision boundary.

In [ ]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

final_metrics = evaluate(model, test_loader, device)
final_auc = auc(*roc_curve(final_metrics['labels'], final_metrics['probs'])[:2])

print(f"""
Dataset
  • Samples:        {len(df)}
  • Sensors:        {len(sensor_cols)} ({", ".join(sensor_cols)})
  • Data Source:    {"Synthetic" if USING_SYNTHETIC_DATA else "Clinical"}

Model
  • Architecture:  Feedforward NN + Autoencoder
  • Structure:     6 → 32 → 16 → 8 → 1 (+ decoder)
  • Regularization Dropout(0.3), L2(0.01), BatchNorm

Training
  • Optimizer:     {chosen_optimizer}
  • Scheduler:     ReduceLROnPlateau
  • Cross-Validation: 5-fold stratified (leakage-free)

Performance
  • Test Accuracy: {final_metrics["accuracy"]:.4f}
  • Test AUC:      {final_auc:.4f}
  • CV Accuracy:   {cv_df["accuracy"].mean():.4f} ± {cv_df["accuracy"].std():.4f}
  • CV AUC:        {cv_df["auc"].mean():.4f} ± {cv_df["auc"].std():.4f}

Representation Quality
  • Mean Recon MSE: {np.mean(recon_errors):.4f}
  • Mean Recon R²:  {np.mean(r2_scores):.4f}
  • Silhouette:     {sil_raw:.3f} → {sil_learned:.3f}
""")

torch.save({
    "model_state_dict": model.state_dict(),
    "scaler_mean": scaler.mean_,
    "scaler_std": scaler.scale_,
    "sensor_cols": sensor_cols,
    "class_names": list(class_names),
    "bottleneck_dim": bottleneck_dim
}, "enose_model.pth")



## What This Project Demonstrates

This work demonstrates that multivariate breath sensor data contain meaningful internal structure that can be uncovered through careful exploratory analysis and effectively exploited through representation learning. Dimensionality reduction reveals substantial redundancy in sensor responses, motivating compact latent representations that capture relevant variation. Within this setting, feedforward neural networks with an autoencoder bottleneck are particularly well suited to the static, low-dimensional, tabular nature of the data, as they learn inter-sensor relationships directly without imposing inappropriate spatial or temporal inductive biases.

The learned latent representations improve class separability relative to the raw sensor space while preserving input structure, as evidenced by increased silhouette scores and high reconstruction fidelity. Model performance remains stable across optimization strategies and stratified cross-validation folds, indicating robustness to training conditions and data partitioning. Interpretability analyses, including reconstruction diagnostics, feature correlations, and gradient-based importance measures, provide insight into how sensor patterns contribute to classification without overclaiming biological specificity. Collectively, these results establish a reproducible and interpretable modeling framework for scent-based health screening under realistic data constraints.

## Limitations and Next Steps

Several limitations warrant consideration. The analysis is based on a single dataset with binary disease labels and a modest sample size, which constrains conclusions regarding generalizability across populations, disease subtypes, and acquisition settings. Although classification performance is consistently near-perfect across stratified cross-validation folds, such results may reflect limited heterogeneity within the study population rather than robust discriminability under real-world conditions. In addition, potentially important confounding factors, including recent diet, smoking status, medications, and comorbidities, were not controlled and may influence breath volatile organic compound (VOC) profiles. Temporal dynamics of sensor responses are not modeled, and sensor drift, an important challenge for electronic-nose deployments, is acknowledged but not explicitly addressed. While reconstruction quality and feature analyses suggest that the learned representations capture physiologically relevant structure, the resulting latent features should be regarded as hypothesis-generating rather than definitive indicators of specific biological mechanisms or biomarkers.

Future work should prioritize external validation using independent cohorts that better reflect population diversity in demographics, disease severity, and clinical context. Integrating electronic-nose sensor arrays with heterogeneous datasets spanning complementary sensing modalities, such as higher-resolution breathomics platforms (e.g., GC–MS–derived VOC profiles) and clinical or wearable data, would enable evaluation of robustness across data sources and acquisition protocols. Beyond classification performance, developing mechanistic interpretability that links learned latent features to specific VOCs or metabolic pathways would strengthen clinical relevance. Incorporating temporal sensor dynamics, extending the framework to multi-class disease phenotyping, and applying calibration or transfer methods to mitigate sensor drift represent additional steps toward practical, deployable healthcare applications.